In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

#每个批次的大小
batch_size = 100
#计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size
num1 = 10 # 第一层多少个卷积核
num2 = 10 # 第二层多少个卷积核
num3 = 10 # 全连接层多少个神经元
#num4 = 10

#初始化权值
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1) #生成一个截断的正态分布
    return tf.Variable(initial)

#初始化偏置值
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

#卷积层
def conv2d(x,W):
    #x input tensor of shape '[batch, in_height, in_width, in_channels]'
    #  tensor 是四维的，批次，长，宽，通道(黑白为1， 彩色为3)
    #W filter /kernel tensor of shape [filter_height, filter_width, in_channels, out_channels]
    #  W 是滤波器，四维， 长，宽，输入，输出
    # stride[0] = stride[3] = 1, strides[1]代表x方向的步长，stride[2]代表y方向的步长
    #  步长
    # padding: A 'string' from: '"SAME", "VALID"'
    # SAME 会补0， VALID不补零
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

#池化层
def max_pool_2x2(x):
    #ksize[1,x,y,1] 窗口大小。
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

# 定义两个placeholder
x = tf.placeholder(tf.float32, [None, 784]) # 28x28
y = tf.placeholder(tf.float32, [None, 10]) 

# 改变x的格式转化为4D的向量[batch, in_height, in_width, in_channels]
x_image = tf.reshape(x, [-1,28,28,1]) # 复原原来的图片

# 初始化第一个卷积层的权值和偏置值
W_conv1 = weight_variable([5,5,1,num1]) # 5x5 的采样窗口， 32个卷积核从1个平面抽取特征
                                      # 1 代表输入通道  32 代表输出32个特征平面
b_conv1 = bias_variable([num1]) # 每一个卷积核一个偏置值

# 把x_image和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1) # 图片和权值传入，它自己就
                                                         # 会算。
h_pool1 = max_pool_2x2(h_conv1) # 进行max_pooling  结果再pooling 

# 初始化第二个卷积层的权值和偏置值
W_conv2 = weight_variable([5,5,num1,num2]) #5x5的采样窗口， 64个卷积核从32个平面抽取特征
b_conv2 = bias_variable([num2]) # 每一个卷积核一个偏置值

# 把h_pool1的权值和向量进行卷积，再加上偏置值，然后用relu激活函数激活
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2) #进行max_pooling

# 28x28 的图片第一次卷积之后还是28x28， 第一次池化之后变成了14x14
# 第二次卷积之后变成14x14， 第二次池化之后变成了7x7
# 经过上面操作后得到64张7x7的平面

#初始化第一个全连接层的权值
W_fc1 = weight_variable([7*7*num2, num3]) #上一层有7*7*64个神经元， 全连接层有1024个神经元
b_fc1 = bias_variable([num3]) # 1024个节点

# 把池化层2的输出层扁平化为1维
h_pool2_flat = tf.reshape(h_pool2, [-1,7*7*num2])
# 求第一个全连接层的输出
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1) + b_fc1)

# keep_prob 用来表示神经元的输出概率
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

# 初始化第二个全连接层
W_fc2 = weight_variable([num3,10]) # 1024个输出给10个输出
b_fc2 = bias_variable([10])

# 计算输出
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2) + b_fc2) # 转化为概率输出

# 交叉熵代价函数
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
# 使用AdamOptimizer进行优化
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
# 结果存放在一个布尔列表中
correct_prediction = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1)) # argmax 返回一维张量中最大值所在位置
# 求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size) # 获取一个批次数据
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7}) # 70% 神经元工作
            
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
        print("Iter " + str(epoch) + " ,Testing Accuracy" + str(acc))
        

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Iter 0 ,Testing Accuracy0.545


KeyboardInterrupt: 